In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
)
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Load the datasets (datasets were already splited and saved as csv files, 60, 20, 20 split)
train_data = pd.read_csv("train_Boarderline_smote_B_data.csv") #train data
test_data = pd.read_csv("test_B_data.csv") #test data
eval_data = pd.read_csv("external_eval_B_data.csv") #external eval data

# Step 2: Separate labels and features
y_train, X_train = train_data.iloc[:, 1], train_data.iloc[:, 2:]
y_test, X_test = test_data.iloc[:, 1], test_data.iloc[:, 2:]
y_eval, X_eval = eval_data.iloc[:, 1], eval_data.iloc[:, 2:]

In [ ]:
# Define Random Forest model and hyperparameter grid
rf_model = RandomForestClassifier(random_state=42) # Random state for reproducibility to get consistent results
param_grid = {
    'n_estimators': [50, 100, 200], # Number of trees in the forest, default = 100, Range = 10-1000
    'max_depth': [10, 20, None], # Maximum depth of the tree, default = None, Range = 1-32, None = full tree
    'min_samples_split': [2, 5, 10],# Minimum number of samples required to split an internal node, default = 2, Range = 2-20
    'min_samples_leaf': [1, 2, 4], # Minimum number of samples required to be at a leaf node, default = 1, Range = 1-20
    'max_features': ['auto', 'sqrt', 'log2'], # Number of features to consider when looking for the best split, default = auto, Range = auto/sqrt/log2
}

# GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='accuracy',
    n_jobs=1,
    cv=5,
    verbose=2
)
grid_search.fit(X_train, y_train)
# Print best parameters and model
print("Best Hyperparameters:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_

# Save best hyperparameters to a file
with open('best_model_info_rf.txt', 'w') as f:
    f.write("Best Hyperparameters:\n")
    for param, value in grid_search.best_params_.items():
        f.write(f"{param}: {value}\n")

print("Best model information has been saved to 'best_model_info_rf_bcell.txt'")

In [ ]:
# Save best hyperparameters to a file
with open('best_model_info_rf_bcell.txt', 'w') as f:
    f.write("Best Hyperparameters:\n")
    for param, value in grid_search.best_params_.items():
        f.write(f"{param}: {value}\n")

print("Best model information has been saved to 'best_model_info_rf_bcell.txt'")

In [ ]:
#save the model
joblib.dump(best_rf_model, "best_rf_bcell_model.pkl")

In [ ]:
# Step 8: Fit the model on the training data
best_rf_model.fit(X_train, y_train)

# Evaluate on test and evaluation datasets
y_test_pred = best_rf_model.predict(X_test)
y_test_prob = best_rf_model.predict_proba(X_test)[:, 1]
y_eval_pred = best_rf_model.predict(X_eval)
y_eval_prob = best_rf_model.predict_proba(X_eval)[:, 1]

# Print accuracies
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred) * 100:.2f}%")
print(f"Evaluation Accuracy: {accuracy_score(y_eval, y_eval_pred) * 100:.2f}%")

# Confusion matrices
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)
confusion_matrix_eval = confusion_matrix(y_eval, y_eval_pred)
print("Confusion Matrix (Test Data):\n", confusion_matrix_test)
print("Confusion Matrix (Evaluation Data):\n", confusion_matrix_eval)

# Sensitivity and Specificity calculation
def calculate_sensitivity_specificity(conf_matrix):
    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

sensitivity_test, specificity_test = calculate_sensitivity_specificity(confusion_matrix_test)
sensitivity_eval, specificity_eval = calculate_sensitivity_specificity(confusion_matrix_eval)
print("Test Data - Sensitivity:", sensitivity_test, "Specificity:", specificity_test)
print("Evaluation Data - Sensitivity:", sensitivity_eval, "Specificity:", specificity_eval)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Generate ROC for test dataset
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_test_prob)
roc_auc_test = auc(fpr_test, tpr_test)

# Generate ROC for evaluation dataset
fpr_eval, tpr_eval, thresholds_eval = roc_curve(y_eval, y_eval_prob)
roc_auc_eval = auc(fpr_eval, tpr_eval)

# Print the values
print("Test Data ROC AUC:", roc_auc_test)
print("Test Data FPR:", fpr_test)
print("Test Data TPR:", tpr_test)
print("Test Data Thresholds:", thresholds_test)

print("Evaluation Data ROC AUC:", roc_auc_eval)
print("Evaluation Data FPR:", fpr_eval)
print("Evaluation Data TPR:", tpr_eval)
print("Evaluation Data Thresholds:", thresholds_eval)

# Create subplots for side-by-side ROC curves
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot Test Data ROC
axes[0].plot(fpr_test, tpr_test, label=f"ROC AUC = {roc_auc_test:.2f}", color="blue")
axes[0].plot([0, 1], [0, 1], 'k--', label="Random Classifier (AUC = 0.50)")
axes[0].set_title("Test Data - ROC Curve")
axes[0].set_xlabel("False Positive Rate")
axes[0].set_ylabel("True Positive Rate")
axes[0].legend(loc="lower right")

# Plot Evaluation Data ROC
axes[1].plot(fpr_eval, tpr_eval, label=f"ROC AUC = {roc_auc_eval:.2f}", color="green")
axes[1].plot([0, 1], [0, 1], 'k--', label="Random Classifier (AUC = 0.50)")
axes[1].set_title("External Validation - ROC Curve")
axes[1].set_xlabel("False Positive Rate")
axes[1].set_ylabel("True Positive Rate")
axes[1].legend(loc="lower right")

# Adjust layout and save figure
plt.tight_layout()
plt.savefig("rf_bcell_roc_auc_curves.png", dpi=500)
print("ROC curves saved as 'rf_roc_auc_curves.png'.")

# Show the plot
plt.show()

In [ ]:
from sklearn.metrics import f1_score
# calculation of F1 score
f1_test = f1_score(y_test, y_test_pred)
f1_eval = f1_score(y_eval, y_eval_pred)
print("F1 Score (Test Data):", f1_test)
print("F1 Score (Evaluation Data):", f1_eval)

In [ ]:
#calculation of classification report for test and evaluation data
print("Classification Report (Test Data):\n", classification_report(y_test, y_test_pred))
print("Classification Report (Evaluation Data):\n", classification_report(y_eval, y_eval_pred))

In [ ]:
# Perform 10-fold cross-validation on the best model
cross_val_scores = cross_val_score(best_rf_model, X_train, y_train, cv=10, scoring='accuracy')
print("10-Fold Cross-Validation Accuracy Scores:", cross_val_scores)
print("Mean 10-Fold CV Accuracy:", cross_val_scores.mean())

# save accuracy of each fold to a text file
with open('cv_accuracies_bcell_rf.txt', 'w') as f:
    f.write("Cross-Validation Accuracies for each fold:\n")
    for i, score in enumerate(cross_val_scores):
        f.write(f"Fold {i+1} Accuracy: {score:.4f}\n")

print("Accuracy of each fold has been saved to 'cv_accuracies_bcell_rf.txt'")

In [ ]:
# make roc curve of all folds with mean auc and mean accuracy printed on the plot
mean_accuracy = cross_val_scores.mean() * 100
print(f"Mean Accuracy: {mean_accuracy:.2f}")
# Step 6: Generate ROC Curve
mean_auc = roc_auc_score(y_train, best_rf_model.predict_proba(X_train)[:, 1])
print(f"Mean AUC: {mean_auc:.4f}")

cv = StratifiedKFold(n_splits=10)
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
for i, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
    best_rf_model.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
    y_prob = best_rf_model.predict_proba(X_train.iloc[test_idx])[:, 1]
    fpr, tpr, _ = roc_curve(y_train.iloc[test_idx], y_prob)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label=f'Fold {i+1} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, label=f'Mean ROC (AUC = {mean_auc:.2f})', lw=2)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig("cv_roc_curve_bcell_rf.png", dpi=500)
plt.show()